# 文生图-Stable Diffusion

https://github.com/open-mmlab/mmagic/tree/main/configs/stable_diffusion

## 指定Prompt文本

In [ ]:
text_prompt1 = 'A panda is having dinner at KFC'

text_prompt2 = 'A Persian cat walking in the streets of New York'

## 方案1. 自定义构造推理模型

In [ ]:

# 提前下载好模型权重，并通过配置文件指定路径

pretrained_model_path = 'data/runway_sd15'

diffusion_scheduler = dict(
    type='EditDDIMScheduler',
    variance_type='learned_range',
    beta_end=0.012,
    beta_schedule='scaled_linear',
    beta_start=0.00085,
    num_train_timesteps=1000,
    set_alpha_to_one=False,
    clip_sample=False)
unet = dict(
    type='UNet2DConditionModel',
    subfolder='unet',
    from_pretrained=pretrained_model_path)
vae = dict(
    type='AutoencoderKL',
    from_pretrained=pretrained_model_path,
    subfolder='vae')

model = dict(
    type='StableDiffusion',
    unet=unet,
    vae=vae,
    enable_xformers=False,
    text_encoder=dict(
        type='ClipWrapper',
        clip_type='huggingface',
        pretrained_model_name_or_path=pretrained_model_path,
        subfolder='text_encoder'),
    tokenizer=pretrained_model_path,
    scheduler=diffusion_scheduler,
    test_scheduler=diffusion_scheduler)

### 实例化模型

In [ ]:
from mmengine import MODELS 
from mmengine.registry import init_default_scope 

init_default_scope('mmagic')

StableDiffuser = MODELS.build(model)
StableDiffuser = StableDiffuser.to('cuda')

### 执行推理

In [ ]:
image = StableDiffuser.infer(text_prompt1)['samples'][0]
image

In [ ]:
image = StableDiffuser.infer(text_prompt2)['samples'][0]
image

## 方案 2. 通过 Inferencer，载入Stable Diffusion模型

网络情况良好时，可尝试该方案

只需要通过指定模型名称来实例化推理接口
初次执行，可能需要一段时间等待模型下载，属于正常情况

In [ ]:
# 构建推理接口

from mmagic.apis import MMagicInferencer
sd_inferencer = MMagicInferencer(model_name='stable_diffusion')


### 执行推理

In [ ]:
image = sd_inferencer.infer(text=text_prompt1, result_out_dir='output/sd_res.png')[1]
image

In [ ]:
image = sd_inferencer.infer(text=text_prompt2, result_out_dir='output/sd_res.png')[1]
image